In [1]:
import numpy as np
import pandas as pd
import os

In [ ]:
data_folder = '../Data'

time_aggregation = 30 # time in minutes. Use -1 to disable the time aggregation

In [2]:
dtype_dict = {
    'STARTTIMEMS': int,
    'ENDTIMEMS': int
}

df = pd.read_csv(f'{data_folder}/2023.csv')#.dropna().astype(dtype_dict)
df.head()

,STARTSTATIONNAME,STARTSTATIONARRONDISSEMENT,STARTSTATIONLATITUDE,STARTSTATIONLONGITUDE,ENDSTATIONNAME,ENDSTATIONARRONDISSEMENT,ENDSTATIONLATITUDE,ENDSTATIONLONGITUDE,STARTTIMEMS,ENDTIMEMS
0,Métro Mont-Royal (Utilités publiques / Rivard),Le Plateau-Mont-Royal,45.524236,-73.581552,Chabot / Mont-Royal,Le Plateau-Mont-Royal,45.534134,-73.573524,1698266696468,1.698267e+12
1,Rielle / Wellington,Verdun,45.460156,-73.567001,St-Jacques / McGill,Ville-Marie,45.501441,-73.560144,1698270535502,1.698272e+12
2,Ste-Catherine / Drummond,Ville-Marie,45.498588,-73.574278,Peel / Ottawa,Le Sud-Ouest,45.494200,-73.559985,1698285705974,1.698286e+12
3,Boyer / du Mont-Royal,Le Plateau-Mont-Royal,45.527432,-73.579917,de l'Hôtel-de-Ville / Rachel,Le Plateau-Mont-Royal,45.519897,-73.580106,1698269489932,1.698270e+12
4,Clark / Ontario,Ville-Marie,45.510625,-73.566903,Wolfe / Robin,Ville-Marie,45.519581,-73.560116,1698270006571,1.698270e+12


Extract and format nodes

In [3]:
df_start = df[['STARTSTATIONNAME', 'STARTSTATIONLATITUDE', 'STARTSTATIONLONGITUDE']].rename(columns={'STARTSTATIONNAME': 'name',
                                                                                                     'STARTSTATIONLATITUDE': 'latitude:float',
                                                                                                     'STARTSTATIONLONGITUDE': 'longitude:float'})
df_end = df[['ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE']].rename(columns={'ENDSTATIONNAME': 'name',
                                                                                             'ENDSTATIONLATITUDE': 'latitude:float',
                                                                                             'ENDSTATIONLONGITUDE': 'longitude:float'})
df_start.head()

,name,latitude:float,longitude:float
0,Métro Mont-Royal (Utilités publiques / Rivard),45.524236,-73.581552
1,Rielle / Wellington,45.460156,-73.567001
2,Ste-Catherine / Drummond,45.498588,-73.574278
3,Boyer / du Mont-Royal,45.527432,-73.579917
4,Clark / Ontario,45.510625,-73.566903


In [4]:
nodes = pd.concat([df_start, df_end], ignore_index=True).drop_duplicates()#.dropna()

print(nodes.shape)

(940, 3)


In [5]:
nodes["stationId:ID"] = nodes.index
nodes[":LABEL"] = "Station"
nodes = nodes[["stationId:ID", "name", "latitude:float", "longitude:float", ":LABEL"]]

In [6]:
nodes.head()

,stationId:ID,name,latitude:float,longitude:float,:LABEL
0,0,Métro Mont-Royal (Utilités publiques / Rivard),45.524236,-73.581552,Station
1,1,Rielle / Wellington,45.460156,-73.567001,Station
2,2,Ste-Catherine / Drummond,45.498588,-73.574278,Station
3,3,Boyer / du Mont-Royal,45.527432,-73.579917,Station
4,4,Clark / Ontario,45.510625,-73.566903,Station


In [7]:
nodes['stationId:ID'] = nodes['stationId:ID'].astype(int)
nodes.to_csv(f'{data_folder}/2023_nodes.csv', index=False, header=False)

In [8]:
import csv
nodes_header = ['stationId:ID','name','latitude:float','longitude:float',':LABEL']
with open(f'{data_folder}/2023_nodes_header.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(nodes_header)

Extract and format edges

In [9]:
# nodes = nodes.drop(columns=[':LABEL'])
# nodes.head()
import datetime

edges = pd.merge(df, nodes,
                 left_on=['STARTSTATIONNAME', 'STARTSTATIONLATITUDE', 'STARTSTATIONLONGITUDE'],
                 right_on=['name', 'latitude:float', 'longitude:float'], how='left')

edges = edges[['ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE', 'STARTTIMEMS', 'ENDTIMEMS', 'stationId:ID']].rename(columns={'stationId:ID': ":START_ID"})

edges = pd.merge(edges, nodes,
                 left_on=['ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE'], 
                 right_on=['name', 'latitude:float', 'longitude:float'], how='left')
# edges.head()
edges = edges[[':START_ID', 'STARTTIMEMS', 'ENDTIMEMS', 'stationId:ID']].rename(columns={'stationId:ID': ":END_ID", 'STARTTIMEMS':'starttimems:int', 'ENDTIMEMS':'endtimems:int'})


to_date = lambda ms: datetime.datetime.utcfromtimestamp(ms/1000).strftime('%Y-%m-%dT%H:%M:%S.%f')
edges['start_date:DATETIME'] = edges['starttimems:int'].apply(to_date)
edges = edges.dropna()
edges[":TYPE"] = "CYCLES_TO"
edges.head()

C:\Users\lpedu\AppData\Local\Temp\ipykernel_18348\554662890.py:18: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  to_date = lambda ms: datetime.datetime.utcfromtimestamp(ms/1000).strftime('%Y-%m-%dT%H:%M:%S.%f')


,:START_ID,starttimems:int,endtimems:int,:END_ID,start_date:DATETIME,:TYPE
0,0,1698266696468,1.698267e+12,509,2023-10-25T20:44:56.468000,CYCLES_TO
1,1,1698270535502,1.698272e+12,301,2023-10-25T21:48:55.502000,CYCLES_TO
2,2,1698285705974,1.698286e+12,471,2023-10-26T02:01:45.974000,CYCLES_TO
3,3,1698269489932,1.698270e+12,424,2023-10-25T21:31:29.932000,CYCLES_TO
4,4,1698270006571,1.698270e+12,1641,2023-10-25T21:40:06.571000,CYCLES_TO


Aggregate per x time

In [ ]:
if time_aggregation > 0:
    df.groupby('category').size().reset_index(name='count')
    edges['starttimems:int'] = edges['starttimems:int'] // (time_aggregation * 60 * 1000)
    edges['endtimems:int'] = edges['endtimems:int'] // (time_aggregation * 60 * 1000)

In [10]:
print(edges.dtypes)

:START_ID                int32
starttimems:int          int64
endtimems:int          float64
:END_ID                  int32
start_date:DATETIME     object
:TYPE                   object
dtype: object


In [11]:
# dtype_mapping = {':START_ID': int, 'starttimems':int, 'endtimems':int, ':END_ID':int, ':TYPE':str}
# edges = pd.DataFrame(edges, dtype=dtype_mapping)
edges[':START_ID'] = edges[':START_ID'].astype('int64')
edges[':END_ID'] = edges[':END_ID'].astype('int64')
edges['starttimems:int'] = edges['starttimems:int'].astype('int64')
edges['endtimems:int'] = edges['endtimems:int'].astype('int64')
edges.to_csv(f'{data_folder}/2023_edges.csv', index=False, header=False)

In [12]:
edges_header = [':START_ID','starttimems:int','endtimems:int',':END_ID','start_date:DATETIME',':TYPE']
with open(f'{data_folder}/2023_edges_header.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(edges_header)

In [13]:
edges.head()

,:START_ID,starttimems:int,endtimems:int,:END_ID,start_date:DATETIME,:TYPE
0,0,1698266696468,1698267092113,509,2023-10-25T20:44:56.468000,CYCLES_TO
1,1,1698270535502,1698271967951,301,2023-10-25T21:48:55.502000,CYCLES_TO
2,2,1698285705974,1698285992418,471,2023-10-26T02:01:45.974000,CYCLES_TO
3,3,1698269489932,1698269814383,424,2023-10-25T21:31:29.932000,CYCLES_TO
4,4,1698270006571,1698270440298,1641,2023-10-25T21:40:06.571000,CYCLES_TO


In [14]:
print(df.shape, "vs.", edges.shape)

(11790840, 10) vs. (11730062, 6)
